In [1]:
import praw
import pandas as pd
import time

# Initializing the Reddit API client with my unique credentials
reddit = praw.Reddit(client_id='9iNmvNiFvQs_V_cu24vphQ',
                     client_secret='wxmYcvLVW7E1dxZJ1HN1VLFkXlTb0A', 
                     password='Reddit!5',
                     user_agent='Jacquelyn', 
                     username='capstoneundercover')

# Specifing the desired subreddit and retrieving the posts
ra_subreddit = reddit.subreddit('RheumatoidArthritis')
post_list = []
for post in ra_subreddit.new(limit=500):  # Retriveing the 500 newest posts at this given time
    post_list.append([post.title, post.selftext, post.score, post.num_comments, post.url])
    if len(post_list) >= 500:  # Stops after retrieving 500 posts to ensure that my data stays accurate and uniform throughout all locations
        break
    time.sleep(2)  # Using a 2-second delay between requests to ensure I don't go over Reddit's API rates
#this was taken on: march 13th, 2023
# Extracting post titles, bodies, scores (upvotes), number of comments, and url
post_titles = []
post_bodies = []
post_scores = []
post_comments = []
post_url = []
for post in post_list:
    post_titles.append(post[0])
    post_bodies.append(post[1])
    post_scores.append(post[2])
    post_comments.append(post[3])
    post_url.append(post[4])

# Creating a DataFrame
df = pd.DataFrame({'post_title': post_titles, 'post_body': post_bodies, 'post_score': post_scores,
                  'post_comments': post_comments, 'post_url': post_url})

# Saving the DataFrame to a CSV file
df.to_csv('ra_posts.csv', index=False)

In [2]:
# Ensuring that the previous code has successfully transferred to my CSV file
# This is not required, but I like the reassurance 
try:
    df.to_csv('ra_posts.csv', index=False)
    print("CSV file successfully saved.")
except Exception as e:
    print("Error occurred while saving CSV:", e)

CSV file successfully saved.


In [6]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    if isinstance(text, str):  # Checking to see if the input is a string
        tokens = word_tokenize(text)  
        tokens = [word.lower() for word in tokens if word.isalpha()]  # Removing punctuation and converting to lowercase
        custom_stop_words = ['got', 'sees', 'today', 'told', 'much', 'we', 'think',
                            'called', 'since']  # Add your custom stop words here
        stop_words.update(custom_stop_words)  # Update the stop_words set with custom stop words
        tokens = [word for word in tokens if word not in stop_words]  # Removing stop words
        return " ".join(tokens)
    else:
        return ""  # Return an empty string for NaN values

# Assuming 'data' is a DataFrame that has been defined earlier
data = df.dropna(subset=['post_body'])  # Removing rows with NaN values in 'post_body' column
data['clean_text'] = data['post_body'].apply(preprocess_text)

data.to_csv('cleaned_ra_data.csv', index=False)

In [9]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv('cleaned_ra_data.csv') # Now you can use the 'df' DataFrame in your code

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# Create an instance of the sentiment analyzer
sentimentAnalyzer = SentimentIntensityAnalyzer()
for post in df.index:
    if isinstance(df['clean_text'][post], str):
        print(df['clean_text'][post][0:100])
        print(sentimentAnalyzer.polarity_scores(df['clean_text'][post]))

gained members january yep least sometimes people say following sub without joining conversation one
{'neg': 0.0, 'neu': 0.748, 'pos': 0.252, 'compound': 0.8683}
hi see posts page see going always pretty athletic like keep healthy really struggling adapt exercis
{'neg': 0.173, 'neu': 0.513, 'pos': 0.314, 'compound': 0.9518}
hi ra friends recently started family planning husband curious anyone tips warnings pregnancies ra a
{'neg': 0.07, 'neu': 0.632, 'pos': 0.299, 'compound': 0.9468}
lots symptoms locking joints intense stiffness worsening muscle weakness sudden constant fatigue neu
{'neg': 0.201, 'neu': 0.625, 'pos': 0.174, 'compound': -0.5043}
diagnosed days ago year untreated ra started severe left back locking knee pain year ago sleep week 
{'neg': 0.225, 'neu': 0.652, 'pos': 0.123, 'compound': -0.9735}
recently diagnosed ra locked right elbow experienced pain daily fully extend elbow currently physica
{'neg': 0.076, 'neu': 0.718, 'pos': 0.206, 'compound': 0.5553}
anyone experience

In [11]:
from textblob import TextBlob

# Replace NaN values in 'clean_text' column with an empty string
df['clean_text'] = df['clean_text'].fillna('')

# Add a new column to the DataFrame to store the sentiment scores
df['sentiment_scores'] = df['clean_text'].apply(lambda text: TextBlob(str(text)).sentiment.polarity)

# Save the updated DataFrame to a new CSV file
df.to_csv('updated_ra_data_with_sentiment.csv', index=False)

In [14]:
import pandas as pd

# Loading the CSV file into the DataFrame again
df = pd.read_csv('updated_ra_data_with_sentiment.csv')

# Exporting the DataFrame to an Excel file located on my computer for easier viewing
df.to_excel('final_ra_posts.xlsx', index=False)